In [2]:
from numba import jit
import numpy as np

In [30]:
@jit(nopython=True, parallel=True)
def a_plus_b(a, b):
    plus = []
    for i, j in zip(a, b):
        plus.append(i+j)
    return plus

In [38]:
def a_plus_b_python(a, b):
    plus = [i+j**2 for i, j in zip(a, b)]
    return plus

In [39]:
n = 1000
a = tuple(range(n))
b = tuple(range(n))

In [32]:
%%timeit
_ = a_plus_b(a, b)

/home/etienne/Projets/py3/lib/python3.5/site-packages/numba/compiler.py:602: NumbaPerformanceWarning: 
The keyword argument 'parallel=True' was specified but no transformation for parallel execution was possible.

To find out why, try turning on parallel diagnostics, see http://numba.pydata.org/numba-doc/latest/user/parallel.html#diagnostics for help.

File "<ipython-input-30-5f8972e62f8b>", line 2:
@jit(nopython=True, parallel=True)
def a_plus_b(a, b):
^

  self.func_ir.loc))


3.53 ms ± 1.22 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [40]:
%%timeit
_ = a_plus_b_python(a, b)

518 µs ± 269 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [21]:
%%timeit
_ = np.array(a) + np.array(b)

69.8 µs ± 2.57 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [45]:
a_nd = np.array(a, dtype=np.float64)
b_nd = np.array(b, dtype=np.float64)

In [46]:
%%timeit
_ = a_nd + b_nd**2

15.2 µs ± 134 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [43]:
from numba import vectorize, float64

@vectorize([float64(float64, float64)])
def f(x, y):
    return x + y**2

In [44]:
%%timeit
_ = f(a_nd, b)

104 µs ± 1.28 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [147]:
import numba

In [155]:
ty = numba.types.Array(np.float64, 1, 'C')

In [150]:
struct_dtype = np.dtype([('xyz', np.float64), ])
ty = numba.from_dtype(struct_dtype)

In [166]:
b = .32
@jit(nopython=True)
def thomas_attractor(xyz):
    ''' ODE for Thomas attractor
        xyz: point positions, shape (dim, )
    '''
    
    sin_xyz = np.sin(xyz)
    dYdt = -b*xyz
    dYdt[0] += sin_xyz[1]
    dYdt[1] += sin_xyz[2]
    dYdt[2] += sin_xyz[0]
    return dYdt

In [167]:
xyz = np.random.rand(3)
_ = thomas_attractor(xyz)#, b=0.19)


In [168]:
%%timeit
_ = thomas_attractor(xyz)#, b=0.19)

# 5.17 µs ± 209 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)  with JIT
# 32.6 µs ± 2.7 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)

# 12.6 µs ± 5.67 µs per loop (mean ± std. dev. of 7 runs, 100000 loops each)  without copy
# 4.68 µs ± 102 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each) without copy and jit

# without optional b
# 3.85 µs ± 185 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)

3.86 µs ± 45.4 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [120]:
b = .32
#@jit(nopython=True)
def thomas_attractor_v2(xyz):
    return np.sin(xyz)[(1, 2, 0), ] - b*xyz

In [129]:
b = .32
@jit(nopython=True)
def thomas_attractor_v3(xyz):
    sin_xyz = np.sin(xyz)
    #dYdt = -b*xyz
    
    dYdt = np.array([sin_xyz[1], sin_xyz[2], sin_xyz[0]]) - b*xyz
    return dYdt

In [133]:
b = .32
@jit(nopython=True)
def thomas_attractor_v4(xyz):
    sin_xyz = np.sin(xyz)
    #dYdt = -b*xyz
    
    #dYdt = np.array([sin_xyz[1], sin_xyz[2], sin_xyz[0]]) - b*xyz
    return (sin_xyz[1] - b*xyz[0],
            sin_xyz[2] - b*xyz[1],
            sin_xyz[0] - b*xyz[2])

In [142]:
xyz = np.random.rand(3)
_ = thomas_attractor(xyz)
print(_)

_ = thomas_attractor_v3(xyz)
print(_)

_ = thomas_attractor_v4(xyz)
print(_)

[ 0.74908454  0.24845246 -0.15178301]
[ 0.74908454  0.24845246 -0.15178301]
(0.7490845414447882, 0.24845246285569217, -0.15178301358607194)


In [144]:
%%timeit
_ = thomas_attractor(xyz)

3.93 µs ± 56.7 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [169]:

@jit(nopython=True)
def normed_thomas_attractor(xyz):
    ''' ODE for Thomas attractor
        xyz: point positions, shape (dim, )
    '''
    
    sin_xyz = np.sin(xyz)
    dYdt = -b*xyz
    dYdt[0] += sin_xyz[1]
    dYdt[1] += sin_xyz[2]
    dYdt[2] += sin_xyz[0]
    return dYdt/np.sqrt(np.sum(dYdt**2))

In [171]:
%%timeit
_ = normed_thomas_attractor(xyz)

4.49 µs ± 363 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
